In [1]:
%scala
val kafka = spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "<Broker IP 1>:9092, <Broker IP 2>:9092, <Broker IP 3>:9092, <Broker IP 4>:9092")
  .option("subscribe", "demo")
  .option("startingOffsets", "earliest")
  .load()

In [2]:
%scala
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val data = kafka
  .withColumn("Key", $"key".cast(StringType))
  .withColumn("Topic", $"topic".cast(StringType))
  .withColumn("Offset", $"offset".cast(LongType))
  .withColumn("Partition", $"partition".cast(IntegerType))
  .withColumn("Timestamp", $"timestamp".cast(TimestampType))
  .withColumn("Value", $"value".cast(StringType))
  .select("Key", "Value", "Partition", "Offset", "Timestamp")

In [3]:
%scala
display(kafka)

In [4]:
%scala
data.writeStream.outputMode("append").format("console").option("truncate", false).start().awaitTermination()

In [5]:
%scala
import java.util.Properties
import org.apache.kafka.clients.producer.KafkaProducer
import org.apache.kafka.clients.producer.ProducerRecord

val topic = "demo"

val props = new Properties()
props.put("bootstrap.servers", "<Server IP 1>:9092, <Server IP 2>:9092, <Server IP 3>:9092, <Server IP 4>:9092")
props.put("acks", "all")
props.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
props.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val producer = new KafkaProducer[String, String](props)

val key = java.util.UUID.randomUUID().toString()
producer.send(new ProducerRecord[String, String](topic, key, "New message")) 
System.out.println("Sent event with key: '" + key + "' and message: '" + "New message" + "'\n")